In [2]:
import math
import os
import pandas as pd
import numpy as np

import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from tqdm import tqdm_notebook

import torch
import torch.nn as nn
import torch.nn.functional as F

train_path = './input/hy_round1_train_20200102'
test_path = './input/hy_round1_testA_20200102'

In [3]:
train_files = os.listdir(train_path)
test_files = os.listdir(test_path)

print(f"训练样本：{len(train_files)}")
print(f"测试样本：{len(test_files)}")

训练样本：7000
测试样本：2000


In [4]:
df = pd.read_csv(f'{train_path}/6966.csv')
print(f"作业类型：{df['type'].unique()}")
print(f"数据维度：{df.shape}")

作业类型：['围网']
数据维度：(389, 7)


In [7]:
def df_angle_handle(dataframe):
    if '方向' in dataframe.columns:
        sin_list = []
        cos_list = []
        for angle in dataframe['方向'].values:
            sin_list.append(math.sin(math.radians(angle)))
            cos_list.append(math.cos(math.radians(angle)))
        dataframe['sin_angle'] = sin_list
        dataframe['cos_angle'] = cos_list
        dataframe = dataframe.drop('方向', axis=1)
    else:
        print("没有方向列或已经处理完毕")
    return dataframe

In [8]:
df = df_angle_handle(df)

In [9]:
df

,渔船ID,x,y,速度,time,type,sin_angle,cos_angle
0,6966,6.265902e+06,5.279254e+06,0.11,1106 23:58:16,围网,-0.809017,0.587785
1,6966,6.265902e+06,5.279254e+06,0.00,1106 23:48:21,围网,0.000000,1.000000
2,6966,6.265902e+06,5.279254e+06,0.00,1106 23:38:19,围网,0.000000,1.000000
3,6966,6.265902e+06,5.279254e+06,0.00,1106 23:28:36,围网,0.000000,1.000000
4,6966,6.265902e+06,5.279254e+06,0.32,1106 23:08:17,围网,0.766044,-0.642788
...,...,...,...,...,...,...,...,...
384,6966,6.312061e+06,5.245699e+06,2.10,1104 00:43:57,围网,0.173648,-0.984808
385,6966,6.312466e+06,5.246134e+06,0.49,1104 00:33:58,围网,-0.798636,-0.601815
386,6966,6.312771e+06,5.246461e+06,1.78,1104 00:23:56,围网,-0.999391,-0.034899
387,6966,6.312974e+06,5.246788e+06,0.49,1104 00:13:57,围网,-0.422618,-0.906308


In [6]:
class LSTM(torch.nn.Module):
    def __init__(self,input_dim, hidden_dim, target_size=3, num_layers=2):
        super(LSTM, self).__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers=2)
        self.fc = nn.Linear(hidden_dim, target_size)
    
    def forward(self, x):
        x, h_n, c_n = self.rnn(x)  # 
        seq_len, batch, h = x.shape  # seq_len, batch, num_directions * hidden_size
        x = x.view(s * b, h)
        x = self.fc(x)
        x = x.view(s, b, -1)
        return x

In [ ]:
model = LSTM(input_dim=)